#### Market Basket Analysis (MBA)
* find relationships between items 
* what items are sold together at a school (flour, sugar) (eggs) 
* basis for recommender engines
* unsupervised learning
* used for data mining, understanding purchasing behaviour
* aka: Affinity Analysis/Cross-sell recommendations 
* typically uses point of sale transaction data/invoices (products purchased at a store)
* can lead to large data set or many unique items(features)
* output is a set of association rules: if item1, item2 then item3
* antecedent (item 1,2) -> consequent (item 3)

Measures of Association
* Support
    * percentage of transactions with a given item combo
    * P(item combo) / all transactions
    * if combo is rare can be dropped
* Confidence
    * condition probability
    * P(buying item B| bought A) = Support (A,B) / support (B)
    * directional
* Lift
    * items A and C are co-occuring in many transactions
    * lift answers if based on chance or real association
    * filtered by support and confidence 
    * Lift (A|B) = support(A,B) / (support(A) * support (B))
    * not directional value of 1+
    * 1 = no association
    * denominator - how often items co-occur if they had NO relationship (A*B)

Python implementation
* uses mlextend package - .frequent_patterns (apriori)
* computational heavy with large amounts of data 


In [ ]:
# python implementation - but no data available 
# will code as comments 

# import pandas as pd
# from mlxtend.frequent_patterns import apriori
# from mlxtend.frequent_patterns import association_rules


# Group by columns to consider 
# market_basket = df[df['Country'] =="United Kingdom"].groupby(
#                 ['InvoiceNo', 'Description'])['Quantity']

# one hot encode the data for mlxtend
# market_basket = market_basket.sum().unstack().reset_index().fillna(0).set_index('InvoiceNo') 
# will end up with a lot of columns (one for each item) and rows with invoice number
# will contain negative numbers - convert negative to 0, and anything above 1 to 1

# #  def encode_data(datapoint):
#     if datapoint <= 0:
#         return 0
#     if datapoint >= 1:
#         return 1

# market_basket = market_basket.applymap(encode_data)

# to find what is sold together use apriori function to data set 
# takes "minimum level of support" - to filter the amount you want - baseline 5%
# support = percentage of time that itemset appears in the data (e.g. 50% = itemset appears 50% of the time)
# too high = few results
# too lower = too many and will need a lot of memory

# itemsets = apriori(market_basket, min_support=0.03, use_colnames=True)

# final step is to set the metric of most interest
# rules = association_rules(itemsets, metric="lift", min_threshold=0.5)

# will result in antecedents and consequents, support, conf, lift etc. 
# can be used to dig a bit deeper to find those rules with high confidence/help to recommend those that occur often

In [1]:
# can be implemented by yourself - since pandas has trouble with large amount of data 

# from itertools import combinations, groupby 
# from collections import Counter

# grabbing only order data
# orders = df_manual.set_index('InvoiceNo')['StockCode']

# find support values for item and frequency
# statistics = orders.value_counts().to_frame("frequency")
# statistics['support']  = statistics / len(set(orders.index)) * 100

# Filter minimum support level
# min_support=0.03 # same value we used above.
# items_above_support = statistics[statistics['support'] >= min_support].index
# orders_above_support = orders[orders.isin(items_above_support)]

# filter out items that were only bought once  in one invoice 
# order_counts = orders.index.value_counts()
# orders_over_two_index = order_counts[order_counts>=2].index
# orders_over_two = orders[orders.index.isin(orders_over_two_index)]

# calculate statistics again
# statistics = orders_over_two.value_counts().to_frame("frequency")
# statistics['support']  = statistics / len(set(orders_over_two.index)) * 100

# Calculate itemsets/itempairs. Function that generates item set 
# calculate freq of each item with each other (freqAC)
# and support (SuppAC)
# filter again below min_support

# def itemset_generator(orders):
#     orders = orders.reset_index().values
#     for order_id, order_object in groupby(orders, lambda x: x[0]):
#         item_list = [item[1] for item in order_object]
#         for item_pair in combinations(item_list, 2):
#             yield item_pair
# itemsets_gen = itemset_generator(orders_over_two)
# itemsets  = pd.Series(Counter(itemsets_gen)).to_frame("frequencyAC")
# itemsets['supportAC'] = itemsets['frequencyAC'] / len(orders_over_two_index) * 100
# itemsets = itemsets[itemsets['supportAC'] >= min_support]

# creating association rules - unstack itemsets - creating datacolumns for support and lift
# Create table of association rules and compute relevant metrics
# itemsets = itemsets.reset_index().rename(columns={'level_0': 'antecedents', 'level_1': 'consequents'})
# itemsets = (itemsets
#      .merge(statistics.rename(columns={'freq': 'freqA', 'support': 'antecedent support'}), left_on='antecedents', right_index=True)
#      .merge(statistics.rename(columns={'freq': 'freqC', 'support': 'consequents support'}), left_on='consequents', right_index=True))

# computing confidence and lift
# itemsets['confidenceAtoC'] = itemsets['supportAC'] / itemsets['antecedent support']
# itemsets['confidenceCtoA'] = itemsets['supportAC'] / itemsets['consequents support']
# itemsets['lift'] = itemsets['supportAC'] / (itemsets['antecedent support'] * itemsets['consequents support'])
# itemsets=itemsets[['antecedents', 'consequents','antecedent support', 'consequents support', 'confidenceAtoC','lift']]

# take only confidence >0.5 
# rules = itemsets
# rules_over_50 = rules[(rules.confidenceAtoC >0.50)]
# rules_over_50.set_index('antecedents',inplace=True)
# rules_over_50.reset_index(inplace=True)
# rules_over_50=rules_over_50.sort_values('lift', ascending=False)


#### Online-Machine Learning
* using data that is streaming
* creme (similar to sklearn), Vowpal Wabbit
* 'get better the more you use it' - usually means scheduled training of new models
* still lags - one week, or day, not 'real time'
* sklearn: partial_fit() gain faster time 
* online learning - new examples are learned from as fast as possible 
* example of  use in cases of emergency in news (where everyone wants at the same time)

Issues
* learning rate too high - will discard things that happened a few seconds ago
* skewed classes problem
* might overfit or underfit 
* harder to scale if updating second to second 
* requires fast access to new data 

In [ ]:
# ### getting time elapsed on training
# import time
# start_time = time.time()
# clf.partial_fit(X, y)
# elapsed_time = time.time() - start_time
# print(elapsed_time)

#### Stochastic Gradient Descent - Review 
* applied to Online Gradient Descent 
* get sum of squares / loss function - plotted get slope and intercept
* adjust slope and intercept by learning rate to get step size
* repeat until max number of steps or convergence
* find derivative of loss function
* used stochastic - takes a random sample - reduce computation time
* learning rate changes from large to small - smaller takes longer to converge
* mini batch - takes a number of random samples - more stable estimates
* new data can be taken to update - takes one more step - from the most recent estimate
* leads to a new 'line' or fit with new data